<a href="https://colab.research.google.com/github/andysingal/machine-learning/blob/main/Ensemble_Learning_and_Random_Forests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

if you aggregate the predictions of a group of predictors (such as classifiers or regressors), you will often get better predictions than with the best individual predictor. A group of predictors is called an ensemble; thus, this technique is called Ensemble Learning, and an Ensemble Learning algorithm is called an Ensemble method.

To make predictions, you obtain the predictions of all the individual trees, then predict the class that gets the most votes , Such an ensemble of Decision Trees is called a Random Forest,